## Imports

In [1]:
import os
import time
import sys
import threading
import boto3
import io
import cv2
from matplotlib import pyplot as plt

In [2]:
def load_images_from_folder(folder):
    imnames = []
    images = []
    for imname in os.listdir(folder):
        imrgb = cv2.imread(os.path.join(folder,imname))
        im = cv2.cvtColor(imrgb, cv2.COLOR_BGR2GRAY)        
        if im is not None:
            imnames.append(imname)
            images.append(im)
    return imnames, images

In [3]:
def upload_image(impath, client, bucket, key):
    client.upload_file(impath, bucket, key)

In [4]:
def upload_image_with_callback(impath, client, bucket, key):
    client.upload_file(impath, bucket, key, Callback=ProgressPercentage(impath))

In [5]:
class ProgressPercentage(object):

    def __init__(self, filename):
        self._filename = filename
        self._size = float(os.path.getsize(filename))
        self._seen_so_far = 0
        self._lock = threading.Lock()

    def __call__(self, bytes_amount):
        # To simplify, assume this is hooked up to a single filename
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = (self._seen_so_far / self._size) * 100
            sys.stdout.write(
                "\r%s  %s / %s  (%.2f%%)" % (
                    self._filename, self._seen_so_far, self._size,
                    percentage))
            sys.stdout.flush()

In [6]:
def put_image(im, client, key, bucket):
    _,encoded = cv2.imencode('.jpg', im)
    im_iobuf = io.BytesIO(encoded)
    client.put_object(Body=im_iobuf, Key=key, Bucket=bucket)

In [7]:
folder='../obstacles_classification_datasets/test_dataset_2'
imnames, images = load_images_from_folder(folder)

In [8]:
bucket = 'obstacles-classification'
key_prefix = 'test_dataset_2/obstacle/'
client = boto3.client("s3")

In [9]:
put_keys = [key_prefix + 'put' + str(i) + '.jpg' for i in range(len(images))]
upload_keys = [key_prefix + 'upload' + str(i) + '.jpg' for i in range(len(images))]

In [10]:
t1 = time.time()

for i in range(len(imnames)):
    impath = os.path.join(folder, imnames[i])
    upload_image(impath, client, bucket, upload_keys[i])
    
upload_total = time.time() - t1

In [11]:
t1 = time.time()

for i in range(len(images)):
    put_image(images[i], client, put_keys[i], bucket)
    
put_total = time.time() - t1

In [12]:
upload_total

21.462606191635132

In [13]:
put_total

19.38068461418152